<a href="https://colab.research.google.com/github/madanmohan-venkatesan/notebooks/blob/main/2011_03567.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
import numpy as np
from scipy.special import gamma

# 2. A sequential multinomial test


In [140]:
#CONSTANTS
CLASSES=2
VECTOR_SIZE=100

In [141]:
theta_null=np.array([1/CLASSES]*CLASSES) #Simplex
rng = np.random.default_rng()
vectors_null=rng.multinomial(1,theta_null,size=VECTOR_SIZE)

In [142]:
K=100
alpha_dirichlet=K*theta_null
rng = np.random.default_rng()
theta_alt=rng.dirichlet(alpha_dirichlet)
vectors_alt=rng.multinomial(1,theta_alt,size=VECTOR_SIZE)

In [143]:
s_n_null=np.sum(vectors_null,axis=0)
s_n_alt=np.sum(vectors_alt,axis=0)

In [144]:
#need some exceptions
def beta(v):
  numerator=np.prod(gamma(v))
  denominator=gamma(np.sum(v))
  return numerator/denominator

In [145]:
#need some exceptions
def bf_10(dirichlet_alpha,theta,counts):
  term1_numerator=beta(dirichlet_alpha+counts)
  term1_denominator=beta(dirichlet_alpha)
  term1=term1_numerator/term1_denominator

  term2=1/(theta**counts)

  return term1*term2

In [146]:
bf_10(alpha_dirichlet,theta_null,s_n_null)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-144-33341291f68e>:4: RuntimeWarning: invalid value encountered in scalar divide
  return numerator/denominator


array([nan, nan])

In [147]:
bf_10(alpha_dirichlet,theta_null,s_n_alt)

<ipython-input-144-33341291f68e>:4: RuntimeWarning: invalid value encountered in scalar divide
  return numerator/denominator


array([nan, nan])

For above bf_10, if theta_0 not equal theta then values will go to infinity (reject null hypothesis)
otherwise value is zero(accept null hypothesis)

Hyper paramters to play around with are K and vector size



In [161]:
theta_null_arr=[]
theta_alt_arr=[]
dirichlet_alpha_arr=[]
vetors_null_arr=[]
vectors_alt_arr=[]
for i in range(10):
  if i==0:
    K=10
  else:
    K=K+(0.5*K)
  #generate null vectors
  theta_null=np.array([1/CLASSES]*CLASSES) #Simplex
  rng = np.random.default_rng()
  vectors_null=rng.multinomial(1,theta_null,size=VECTOR_SIZE)
  #generate alt vectors

  alpha_dirichlet=K*theta_null
  rng = np.random.default_rng()
  theta_alt=rng.dirichlet(alpha_dirichlet)
  vectors_alt=rng.multinomial(1,theta_alt,size=VECTOR_SIZE)

  theta_null_arr.append(theta_null)
  theta_alt_arr.append(theta_alt)
  dirichlet_alpha_arr.append(alpha_dirichlet)
  vetors_null_arr.append(vectors_null)
  vectors_alt_arr.append(vectors_alt)

In [162]:
#give some exceptions
on_theta_hist_arr=[]
for i in range(len(theta_null_arr)):
  on_theta_prev = None
  if i==0:
    on_theta_prev = np.array([1,1])
    on_theta_hist_arr.append(on_theta_prev)

  else:
    s_n_null=np.sum(vetors_null_arr[i],axis=0)
    s_n_alt=np.sum(vectors_alt_arr[i],axis=0)
    term1_numerator=beta(dirichlet_alpha_arr[i]+s_n_alt)
    term1_denominator=beta(dirichlet_alpha_arr[i])
    term1=term1_numerator/term1_denominator

    term2=1/(theta_null_arr[i]**s_n_alt)

    on_theta_hist_arr.append(on_theta_hist_arr[i-1]*term1*term2)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-144-33341291f68e>:4: RuntimeWarning: invalid value encountered in scalar divide
  return numerator/denominator


In [163]:
on_theta_hist_arr=np.array(on_theta_hist_arr)
on_theta_hist_arr

array([[1.00000000e+00, 1.00000000e+00],
       [2.96210777e-15, 4.62829339e-17],
       [1.03667187e-29, 2.53093717e-33],
       [7.52864345e-47, 3.01145738e-46],
       [9.53014750e-60, 2.32669617e-63],
       [0.00000000e+00, 0.00000000e+00],
       [           nan,            nan],
       [           nan,            nan],
       [           nan,            nan],
       [           nan,            nan]])

In [164]:
np.where(on_theta_hist_arr<(1/0.05),on_theta_hist_arr,np.inf)

array([[1.00000000e+00, 1.00000000e+00],
       [2.96210777e-15, 4.62829339e-17],
       [1.03667187e-29, 2.53093717e-33],
       [7.52864345e-47, 3.01145738e-46],
       [9.53014750e-60, 2.32669617e-63],
       [0.00000000e+00, 0.00000000e+00],
       [           inf,            inf],
       [           inf,            inf],
       [           inf,            inf],
       [           inf,            inf]])

# Optimization problem for theta

In [168]:
# idx=np.argwhere(on_theta_hist_arr<(1/0.05))
# # on_theta_hist_arr[idx].shape
# idx

on_theta_hist_arr[on_theta_hist_arr<(1/0.05)].shape

(12,)

In [156]:
c = np.log(beta(alpha_dirichlet+s_n_alt)) - np.log(beta(alpha_dirichlet))
u=0.05

<ipython-input-144-33341291f68e>:4: RuntimeWarning: invalid value encountered in scalar divide
  return numerator/denominator


In [ ]:
def constraint_1(theta,c,u,s_n_alt):
  return c+np.log(u)-np.sum(s_n_alt*np.log(theta))
def constraint_2(theta):
  return np.sum(theta)-1